This MATLAB program simulates a mobile robot navigating a 10×10 grid while avoiding obstacles and reaching multiple goal points. The robot uses the A algorithm* for pathfinding and a Bézier curve for smooth trajectory generation. It includes reactive obstacle avoidance and a kinematic model for motion.

**1. Initialization**

In [ ]:
clc;
clear;
close all;


*   clc; → Clears the command window.
*   clear; → Clears all variables from the workspace.
*   close all; → Closes all open figure windows.




**2. Map and Parameter**

In [ ]:
mapSize = 10; % 10x10 grid
numGoals = 3; % Number of goal points
robotRadius = 0.2; % Robot size (radius)
obstacleDensity = 0.15; % Fraction of the area occupied by obstacles
robotSpeed = 0.2; % Robot speed
robotAngle = 0; % Initial robot orientation


*   Defines the map size *(10×10)*.
*   Sets three goal points.
*   Defines robot parameters *(size, speed, and angle)*.
*   obstacleDensity defines the fraction of the grid occupied by obstacles.




**3. Map Initialization**

In [ ]:
map = zeros(mapSize, mapSize);
obstacles = rand(mapSize, mapSize) < obstacleDensity;
goals = generateRandomGoals(numGoals, mapSize, obstacles);


*   *map = zeros(mapSize, mapSize);* initializes an empty map.
*   *rand(mapSize, mapSize) < obstacleDensity;* randomly places obstacles.
*   *generateRandomGoals(...)* generates goal points that are not occupied by obstacles.





Visualization Setup

**4. Setting Up the Figure**

In [ ]:
figure;
hold on;
axis([0 mapSize 0 mapSize]);
grid on;
set(gca, 'xtick', 0:mapSize);
set(gca, 'ytick', 0:mapSize);
plotGoalsAndObstacles(goals, obstacles);



*   Creates a grid for visualization.
*   Plots goals and obstacles on the map.



**5. Robot Initialization**

In [ ]:
robotPos = [1, 1]; % Start at (1,1)
odometryPos = robotPos; % Odometry tracks estimated position
robotPlot = plot(robotPos(1), robotPos(2), 'bo', 'MarkerSize', 8, 'MarkerFaceColor', 'b');
pathLine = plot(robotPos(1), robotPos(2), 'b-', 'LineWidth', 1.5);
pathPoints = robotPos;


*   Sets initial position at *(1,1)*.
*   Plots the robot as a blue circle *('bo')*.
*   Tracks the path using *pathLine*.



**6. Navigation Through Goals**

In [ ]:
for goalIdx = 1:numGoals
    [robotPos, odometryPos, robotAngle, pathPoints] = navigateToGoal(robotPos, goals(goalIdx, :), obstacles, robotSpeed, mapSize, robotPlot, robotRadius, goalIdx, robotAngle, odometryPos, pathPoints, pathLine);
end
disp('Simulation complete!');


*   Iterates through each goal point and calls *navigateToGoal()*.
*   The robot updates its position, angle, and odometry at each step.

Goal Generation

**generateRandomGoals() Function**

In [ ]:
function goals = generateRandomGoals(numGoals, mapSize, obstacles)
    goals = zeros(numGoals, 2);
    for i = 1:numGoals
        while true
            randX = randi(mapSize);
            randY = randi(mapSize);
            if obstacles(randX, randY) == 0
                goals(i, :) = [randX, randY];
                break;
            end
        end
    end
end


*   Generates random goals while avoiding obstacles.


**Pathfinding (A)***

aStarPathfinding() Function

In [ ]:
function path = aStarPathfinding(startPos, goal, obstacles, mapSize)
    openSet = containers.Map;
    openSet(num2str(startPos)) = heuristic(startPos, goal);
    cameFrom = containers.Map;
    gScore = containers.Map('KeyType', 'char', 'ValueType', 'double');
    gScore(num2str(startPos)) = 0;

    while ~isempty(openSet)
        [currentPos, ~] = minKey(openSet);
        openSet.remove(currentPos);
        currentPos = str2num(currentPos);

        if isequal(currentPos, goal)
            path = reconstructPath(cameFrom, currentPos);
            return;
        end

        neighbors = getNeighbors(currentPos, mapSize);
        for i = 1:size(neighbors, 1)
            neighbor = neighbors(i, :);
            if neighbor(1) < 1 || neighbor(1) > mapSize || neighbor(2) < 1 || neighbor(2) > mapSize
                continue;
            end
            neighbor = round(neighbor);
            if obstacles(neighbor(1), neighbor(2)) == 1
                continue;
            end
            tentativeGScore = gScore(num2str(currentPos)) + 1;
            if ~isKey(gScore, num2str(neighbor)) || tentativeGScore < gScore(num2str(neighbor))
                cameFrom(num2str(neighbor)) = currentPos;
                gScore(num2str(neighbor)) = tentativeGScore;
                openSet(num2str(neighbor)) = gScore(num2str(neighbor)) + heuristic(neighbor, goal);
            end
        end
    end
    path = [];
end

A Algorithm Explanation*

*   Open Set (openSet): Stores nodes to be explored, prioritized by f = g + h.
*   G-Score (gScore): Stores cost from start to node.
*   Came From (cameFrom): Tracks path reconstruction.
* Loop Until Goal Found:

Selects node with the lowest f-score.
Generates valid neighbors.
Updates scores if a better path is found.



**heuristic() Function (Euclidean Distance)**

In [ ]:
function h = heuristic(pos, goal)
    h = sqrt((pos(1) - goal(1))^2 + (pos(2) - goal(2))^2);
end


*   Uses Euclidean distance as a heuristic.



**Path Reconstruction**

In [ ]:
function path = reconstructPath(cameFrom, currentPos)
    path = currentPos;
    while isKey(cameFrom, num2str(currentPos))
        currentPos = cameFrom(num2str(currentPos));
        path = [currentPos; path];
    end
end


*   Traces back from the goal to reconstruct the path.

**Path Smoothing**

Bézier Curve Smoothing

In [ ]:
function smoothPath = bezierSmoothPath(path)
    t = linspace(0, 1, 100);
    n = size(path, 1) - 1;
    smoothPath = zeros(length(t), 2);
    for i = 0:n
        B = nchoosek(n, i) .* (t .^ i) .* ((1 - t) .^ (n - i));
        smoothPath = smoothPath + B' * path(i + 1, :);
    end
end


*   Generates a Bézier curve to smooth out sharp turns.



**Reactive Obstacle Avoidance**

In [ ]:
function robotPos = reactiveAvoidance(robotPos, obstacles, robotSpeed, mapSize, robotPlot, robotRadius)
    proximityRadius = robotRadius + 1;
    [xObst, yObst] = find(obstacles);
    for i = 1:length(xObst)
        dist = sqrt((xObst(i) - robotPos(1))^2 + (yObst(i) - robotPos(2))^2);
        if dist < proximityRadius
            angleToObstacle = atan2(yObst(i) - robotPos(2), xObst(i) - robotPos(1));
            avoidanceAngle = angleToObstacle + pi / 2;
            robotPos = robotPos + [cos(avoidanceAngle) * 0.5, sin(avoidanceAngle) * 0.5];
            break;
        end
    end
end


*   Moves the robot away from nearby obstacles.

